In [2]:
import os
import pandas as pd
import ast
pd.set_option('display.max_columns', None)

import numpy as np

In [5]:
# CONSTANTS

data_path = r"./data/interm/data_to_train_meantime.csv"

In [33]:
# LOAD THE ESSENTIALS

df = pd.read_csv(data_path)

In [34]:
df = df.drop(columns=["Unnamed: 0","submit_time"])

In [36]:
df

,submit_day,submit_hour,group_id,num_tasks,cores_per_task,num_nodes_req,num_cores_req,req_nodes,req_switch,num_gpus_req,mem_req,job_mean_power_consumption,job_min_power_consumption,job_max_power_consumption
0,31,22,25200,64.0,4,16,256,0,0,64,475,519.930556,465.00,531.250
1,31,22,25200,64.0,4,16,256,0,0,64,475,510.301724,363.75,533.125
2,31,22,25200,64.0,4,16,256,0,0,64,475,512.069444,362.50,531.875
3,31,23,25200,0.0,32,1,32,0,0,4,59,630.377358,520.00,860.000
4,31,23,25200,0.0,32,1,32,0,0,4,59,860.208333,860.00,870.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231233,7,5,25200,4.0,32,1,128,0,0,4,237,920.000000,920.00,920.000
231234,7,7,25200,4.0,1,1,4,0,0,4,7,863.000000,860.00,870.000
231235,7,9,25200,8.0,16,1,128,0,0,4,234,877.745455,610.00,1000.000
231236,7,6,25200,4.0,32,1,128,0,0,4,237,860.000000,860.00,860.000


In [49]:
columns_set = set(df.columns.values.tolist())
target_set = set(["job_mean_power_consumption","job_min_power_consumption","job_max_power_consumption"])
Y_columns = list(target_set)
X_columns = list(columns_set - target_set)

In [50]:
df[X_columns]

,num_nodes_req,num_tasks,submit_hour,group_id,num_cores_req,mem_req,num_gpus_req,req_nodes,submit_day,cores_per_task,req_switch
0,16,64.0,22,25200,256,475,64,0,31,4,0
1,16,64.0,22,25200,256,475,64,0,31,4,0
2,16,64.0,22,25200,256,475,64,0,31,4,0
3,1,0.0,23,25200,32,59,4,0,31,32,0
4,1,0.0,23,25200,32,59,4,0,31,32,0
...,...,...,...,...,...,...,...,...,...,...,...
231233,1,4.0,5,25200,128,237,4,0,7,32,0
231234,1,4.0,7,25200,4,7,4,0,7,1,0
231235,1,8.0,9,25200,128,234,4,0,7,16,0
231236,1,4.0,6,25200,128,237,4,0,7,32,0


In [64]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# MAKE THE SPLITS

X,y = df[X_columns], df[Y_columns]

### FOR CLASSIC MODELS

# Assuming you have your data in X (features) and y (labels)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit only on train!
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)


### FOR TORCH MODELS

# Convert to tensors (on CPU)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # Use float32 for regression

X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import wandb

In [14]:
import random

import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="iqbalch-universidad-carlos-iii-de-madrid",
    # Set the wandb project where this run will be logged.
    project="Test",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 20
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

acc,▃▁▆▅▆▆▆▇▇▇▇▇▇█████
loss,█▆█▂▄▂▃▃▃▂▂▃▁▂▁▁▂▁
acc,0.93385
loss,0.06768
